In [1]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, models, losses
from sentence_transformers.evaluation import LabelAccuracyEvaluator
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import os

# Optional: Set which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1)) 

1
NVIDIA A100-SXM4-80GB


AssertionError: Invalid device id

## Preparing the dataset

In [4]:
from datasets import load_dataset
from sentence_transformers import InputExample

# Load both train and validation splits
hebnli = load_dataset('HebArabNlpProject/HebNLI')

ENTAILMENT = "entailment"
CONTRADICTION = "contradiction"
PREMISE = "translation1"
HYPOTHESIS = "translation2"
LABEL = "original_label"

# Use only 'entailment' pairs (label==2) for MultipleNegativesRankingLoss
def make_examples(dataset, include_negative=False):
    examples = [
        InputExample(texts=[item[PREMISE], item[HYPOTHESIS]], label=1.0)
        for item in dataset
        if item[LABEL] == ENTAILMENT and item[PREMISE] and item[HYPOTHESIS]
    ]
    if include_negative:
        examples += [
            InputExample(texts=[item[HYPOTHESIS], item[PREMISE]], label=0.0)
            for item in dataset
            if item[LABEL] == CONTRADICTION and item[PREMISE] and item[HYPOTHESIS]
        ]
    return examples

train_examples = make_examples(hebnli['train'])
val_examples   = make_examples(hebnli['dev'], include_negative=True)

print(f"Train: {len(train_examples)}, Val: {len(val_examples)}")


Train: 100995, Val: 1311


In [6]:
train_dataloader = DataLoader(train_examples, shuffle=True,  batch_size=128)

## Model setup

In [7]:
model_name_or_path = "/home/nlp/achimoa/workspace/ModernBERT/hf/HebrewModernBERT/ModernBERT-Hebrew-base_20250522_0255"
bert = models.Transformer(
    model_name_or_path, 
    max_seq_length=1024,
)
pooling = models.Pooling(bert.get_word_embedding_dimension())
sbert_model = SentenceTransformer(modules=[bert, pooling])

## Loss function

In [8]:
train_loss = losses.MultipleNegativesRankingLoss(sbert_model)

## Validation

In [9]:
sents1 = [ex.texts[0] for ex in val_examples]
sents2 = [ex.texts[1] for ex in val_examples]
labels = [ex.label for ex in val_examples]

val_evaluator = EmbeddingSimilarityEvaluator(sents1, sents2, labels, name='validation')

## Training

In [10]:
sbert_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=val_evaluator,
    evaluation_steps=10,
    epochs=1,
    output_path='./outputs/models/sbert/sbert-hebmodernbert-snli',
    save_best_model=True,
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/790 [00:00<?, ?it/s]

KeyboardInterrupt: 